## Objectives:
1. Take input (clusters)

2. Remove noisey clusters (outliers, spam: weather reports, amazon, etc.)

3. Randomly sample 100 text entries from relevant cluster(s)

4. Run through GPT classification prompt. 

5. Store outputs

6. Analyze label distributions

In [8]:
pip install openai -q


SyntaxError: invalid syntax (3219844315.py, line 1)

In [9]:
pip install python-dotenv -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
# Imports
import os
import pandas as pd
import openai
import pandas as pd
import time
import pickle
from dotenv import load_dotenv

In [20]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [26]:
# Input

# Note: Current input uses clusters from Run1: 3 topics

input_path = "/Users/tobiasmichelsen/Bachelor_Project/DS_BachelorProject_PH/GPT/all_clusters.json"

df = pd.read_json(input_path,lines=True)
df_cluster0 = df[df.topic==0]
sampled_df = df_cluster0.sample(n=min(100, len(df_cluster0)), random_state=42).reset_index(drop=True)

In [27]:
print(sampled_df)

           seq                                               text  topic
0   7845056826  UN, Germany say tackling climate crisis is pat...      0
1   7834751068  One of my favorite youtubers that I have watch...      0
2   8164487717  There is no 'controversey' over efficiency sta...      0
3   8051802123  Update on the bucket system. So far so good, j...      0
4   8094293684  tumblr.comreblogfanfi... HazbinHotel Alastor H...      0
..         ...                                                ...    ...
95  8180397464  The US government is preparing executive order...      0
96  7947942100  The UK exports tons of ferrous scrap because i...      0
97  7959037306  The Greens are bringing bold ideas like Guaran...      0
98  8066135003  detection could be unlocked across a range of ...      0
99  8093921828  This morning we celebrated the opening of the ...      0

[100 rows x 3 columns]


In [30]:
multi_label_prompt_template = """
You are a helpful assistant that classifies text into one or more climate-related categories.

Use the examples below to guide your classification:

- Activism: "Thousands marched in the climate strike demanding government action.", "Youth-led campaigns are pressuring lawmakers to enact stronger climate policies.", "Environmental activists chained themselves to construction equipment to protest deforestation."
- Agriculture: "Crops are failing due to prolonged droughts intensified by climate change.", "Pollinators like bees are disappearing, threatening global food security.", "Rising temperatures are affecting livestock health and reducing dairy production on farms."
- Disaster: "Wildfires have devastated thousands of acres in California.", "Flooding from intense storms has displaced hundreds of families across the region.", "A powerful hurricane made landfall, causing widespread destruction and power outages."
- Fossil: "Oil prices continue to rise amid geopolitical tensions and supply constraints.", "New coal-fired power plants are being built despite international climate agreements.", "Natural gas usage has surged as countries transition away from coal and nuclear energy."
- Lifestyle: "People are embracing minimalist living to reduce their carbon footprint.", "Plant-based diets are gaining popularity for their environmental benefits.", "Air pollution is linked to rising asthma rates in urban areas."
- Politics: "The new administration reversed several environmental protections.", "Communities of color are disproportionately affected by environmental hazards.", "Lawmakers are debating a new bill aimed at cutting national carbon emissions by 2030."
- Renewable: "Government subsidies are making rooftop solar panels more accessible.", "Community wind projects are helping rural areas become energy independent.", "Drought conditions are affecting the electricity output of hydropower plants in Brazil."
- Waste: "Cities are expanding composting and recycling programs to reduce landfill use.", "Single-use plastics are being banned in several countries to combat environmental pollution.", "Innovative startups are turning food waste into sustainable packaging materials."
- Weather: "Global temperatures hit a new record high this year.", "An unprecedented heatwave swept across Europe, breaking temperature records.", "Heavy rainfall and flash floods have disrupted transportation in the region."

If the text belongs to more than one category, list all relevant categories separated by commas.
If the text does not fit any category above, assign a new category name.

Classify the following text:
"{text}"

Response:
"""




In [29]:
def run_classification(df, prompt_template, label_type="multi"):
    results = []

    for idx, row in df.iterrows():
        text = row["text"]
        seq = row["seq"]
        prompt = prompt_template.format(text=text)

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.0,
                max_tokens=50
            )
            label = response["choices"][0]["message"]["content"].strip()

        except Exception as e:
            label = f"ERROR: {str(e)}"

        results.append({
            "seq": seq,
            "text": text,
            "label_type": label_type,
            "gpt_response": label
        })

        if idx % 10 == 0:
            print(f"[{label_type}] Processed {idx+1} samples")

    return pd.DataFrame(results)

In [ ]:
multi_df = run_classification(sampled_df, multi_label_prompt_template, label_type="multi")
#single_df = run_classification(sampled_df, single_label_prompt_template, label_type="single")

# Save results
multi_df.to_csv("/Users/tobiasmichelsen/Bachelor_Project/DS_BachelorProject_PH/GPT/artifacts/gpt_cluster0_multilabel.csv", index=False)
#single_df.to_csv("gpt_cluster0_singlelabel.csv", index=False)
